In [1]:
import numpy as np
import torch
import torchvision
from typing import Tuple

# Sieci neuronowe

## MNIST
Popularnym "prostym" datasetem, na którym można przetestować nasz model zanim zajmiemy się trudniejszymi problemami, jest MNIST, zbiór danych zawierających ręcznie rysowane cyfry. Poniżej kilka przykładowych cyfr:

<img width="400" src="https://miro.medium.com/proxy/0*At0wJRULTXvyA3EK.png" />


Zadanie naszego modelu polega na tym, by na podstawie obrazka narysowanej ręcznie cyfry określić, jaka to jest cyfra.

Czyli nasz model, widząc taki obrazek:
<img width="200" src="https://machinelearningmastery.com/wp-content/uploads/2019/02/sample_image-768x763.png" />

powinien odpowiedzieć, że to cyfra "7".


**Pytanie: Czy w takim sformułowaniu MNIST służy do regresji czy do klasyfikacji?**

## Praca na obrazkach
Nasze modele jak dotąd przyjmowały wyłącznie wektory - w jaki sposób możemy w takich modelach przetwarzać obrazki?

Obrazek to tak naprawdę trójwymiarowa tablica pikseli o wymiarach: `[H, W, C]`, gdzie `H` to wysokość, `W` to szerokość, a `C` to liczba kanałów (klasycznie: red, green, blue).

Najprostsze co możemy zrobić, to spłaszczyć naszą tablicę do jednego wymiaru, wektora o kształcie `[H * W * C]`. W przypadku MNIST-a nasze obrazki mają wymiar `28x28` pikseli i jest tylko jeden kanał (odcienie szarości), więc każdy z naszych wektorów będzie miał kształt `[28 * 28] = [784]`.

W przyszłości poznamy też sprytniejsze sposoby działania na obrazkach, np. za pomocą sieci konwolucyjnych.

## Stochastic gradient descent

Dotychczas kiedy chcieliśmy minimalizować funkcję kosztu $L(X; \theta)$ dla całego naszego zbioru $X \in \mathbb{R}^{NxD}$, liczyliśmy średni koszt dla wszystkich elementów $x \in X$, tzn. 

$$L(X; \theta) = \frac{1}{N} \sum_i L(x_i; \theta) $$

Następnie liczyliśmy gradient tego kosztu, żeby zminimalizować funkcję.

W praktyce może się okazać, że nasz dataset jest gigantyczny, np. kiedy mamy miliony przykładów. Niepraktyczne wtedy jest liczenie całego tego kosztu a tym bardziej gradientu. W praktyce w każdym kroku liczymy funkcję kosztu (i jej gradient) z innego podzbioru elementów w naszym zbiorze, czyli z tzw. **batcha**:

$$L_{\mathrm{batch}} (X;\theta) = \frac{1}{|B|} \sum_{x \in B} L(x; \theta) $$ 

Gradient po koszcie policzonym z batcha będzie inny niż gradient liczony po koszcie policzonym z całego zbioru, ale powinny być w miarę podobne, tzn:

$$ \nabla_\theta L_{\mathrm{batch}} (X; \theta) \approx \nabla_\theta L(X; \theta) $$

Metodę spadku gradientu zaimplementowaną w ten sposób (batchowo) nazywamy metodą **stochastycznego spadku gradientu** (*Stochastic Gradient Descent, SGD*). 


## Torchvision
PyTorch, a także pakiet `torchvision` udostępnia parę przydatnych narzędzi, z których skorzystamy na dzisiejszych zajęciach. Dla przykładu znacznie uproszczone jest pobieranie i ładowanie danych. W pakiecie [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html) znajdziemy popularne datasety, m.in. właśnie MNIST-a.

Oprócz tego z samego `torcha` możemy skorzystać z [`DataLoadera`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), który implementuje wiele przydatnych operacji do ładowania danych, np. dzielenie datasetu w batche i shufflowanie.

## Zadanie 1 (2 pkt.)
Klasa `MNIST` zwraca nam dane w postaci obiektów [PIL](https://pillow.readthedocs.io/en/stable/). Należy je odpowiednio przetworzyć, zanim będziemy mogli na nich pracować.

Za pomocą [`transformacji`](https://pytorch.org/vision/stable/transforms.html) podawanych do klasy MNIST należy:
1. Zamienić obiekty `PIL` na Tensory.
2. Policzyć średnią i odchylenie standardowe pikseli dla całego zbioru trenującego i użyć ich później do znormalizowania danych trenujących i testowych. Do liczenia średniej i odchylenia standardowego wykorzystać funkcję  `calculate_mean_and_std` (proszę zwrócić uwagę na to w jakim przedziale znajdują się dane przed normalizacją – chcemy aby były w przedziale 0-1). **HINT**: Tutaj torchvision powinien nam to ułatwić.
3. Zmienić "kształt" każdego przykładu z `28x28` na `784`.  
    **HINT**: [`Lambda`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Lambda)

Uwaga: proszę zwrócić uwagę co dokładnie robią używane_transformacje!

In [9]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Lambda, Compose, Normalize

def calculate_mean_and_std() -> Tuple[float, float]:
    mean = torch.mean(train_data[0][0])
    std = torch.std(train_data[0][0])
    print(mean, std)
    return mean, std

# czyli normalizacji nie mogę robić kiedy otrzymuję dane, bo potrzebuje ona mean i std (lub nie potrzebuje)
# w każdym razie dane normalizuję po tym jak je wyciągnę i zwrócę mean i std

# def normalize(data, mean, std):
#     print("2nd")
#     return (data - mean) / std

print("1st")

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
#         torchvision.transforms.Lambda(calculate_mean_and_std)                                       #lambda x: normalize(x, mean, std)),
        torchvision.transforms.Lambda(lambda x: calculate_mean_and_std(train_data[0][0])),
        torch.flatten
    ]
)


train_data = MNIST(
    root='.',
    download=True,
    train=True, 
    transform=transform
)

test_data = MNIST(
    root='.', 
    download=True, 
    train=False, 
    transform=transform
)

mean, std = calculate_mean_and_std()
norm_data = Normalize(mean=mean, std=std)
# train_data[0] = torch.nn.functional.normalize(train_data[0][0]), train_data[0][1]
# # print(train_data.shape)
# train_data[0] = torch.flatten(train_data), train_data[0][1]
# print(train_data.shape)


1st
tensor(0.1377) tensor(0.3125)


In [10]:
mean, std = calculate_mean_and_std()
assert np.isclose(mean, 0.1306, atol=1e-4)
assert np.isclose(std, 0.3081, atol=1e-4)

tensor(0.1377) tensor(0.3125)


AssertionError: 

In [11]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=10)

x, y = next(iter(train_loader))
assert len(x.shape) == 2
assert x.shape == (10, 784)

NameError: name 'mean' is not defined

# Sieci neuronowe

### Modele liniowe

Jak dotąd omawialiśmy wyłącznie modele liniowe, tzn. takie, które dla zadanego $x$ potrafiły modelować funkcję rodzaju $$f(x) = g(w^T x + b) $$
gdzie $x \in \mathbb{R}^D, w \in \mathbb{R}^D$, $b \in \mathbb{R}$ a $g$ to funkcja aktywacji, np. sigmoid.

Możemy też stworzyć podobny model, który na wyjściu nie będzie podawał jednej liczby, ale cały wektor o wymiarze $K$, tzn:
$$f(x) = g(W^T x + \mathbf{b}), $$
gdzie $W$ jest teraz macierzą a $\mathbf{b}$ wektorem, tzn. $W \in \mathbb{R}^{DxK}, \mathbf{b} \in \mathbb{R}^{K}$.

### Zanurzenia

Jeżeli chcieliśmy, żeby takie modele mogły zajmować się problemami nieliniowymi, musieliśmy znaleźć odpowiednią reprezentację danych (zanurzenia wielomianowe, kernele dla SVM-ów), które sprawi, że w nowej przestrzeni problem będzie liniowy. W tym celu trzeba "zgadnąć", jakie przekształcenie jest właściwe - co w przypadku bardziej skomplikowanych problemów jest niezwykle trudne. 

Ważne pytanie: **czy jesteśmy w stanie zbudować model, który znajdzie nam odpowiednią reprezentację dla danych?**

### Nakładanie warstw liniowych

**Rozwiązanie:** Nałóżmy na siebie kilka warstw modeli liniowych, np:
$$
f(x) = g_2(W_2^T (g_1(W_1^T x + \mathbf{b_1})) + \mathbf{b_2}),
$$
czyli, rozpisując czytelniej:
$$
f(x) = f^{(2)}(f^{(1)}(x)) \\
f^{(1)}(x) = g_1(W_1^T x + \mathbf{b_1}) \\ 
f^{(2)}(x) = g_2(W_2^T x + \mathbf{b_2})
$$

Powstały model nazywamy **sztuczną siecią neuronową** (*artificial neural network*).
 
Każdą funkcję $f^{(i)}$ nazywamy **warstwą** (*layer*). W naszej sieci możemy umieścić dowolnie wiele warstw, ale na razie będziemy zajmować się modelami nieszczególnie głębokimi (mniej niż 10 warstw).

Warstwy $f^{(i)}$ mogą implementować dowolną funkcję, ale jeśli mają postać $g(W^Tx +\mathbf{b})$, to nazywamy je warstwami liniowymi lub warstwami *fully connected*. Na tych zajęciach będziemy zajmować się wyłącznie sieciami o takiej postaci.


### Uczenie się reprezentacji

Jeśli nasz model jest postaci
$$
f(x) = f^{(n)}(f^{(n-1)}(\ldots f^{1}(x) \ldots )),
$$
to możemy przyjąć, że warstwa $f^{(n)}$ rozwiązuje problem liniowy na reprezentacji zadanej przez warstwy $f^{(1)}, f^{(2)}, \ldots, f^{(n-1)}$. 



<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Colored_neural_network.svg/1200px-Colored_neural_network.svg.png" width=300 />  

Źródło: [Wikipedia](https://en.wikipedia.org/wiki/Artificial_neural_network).

### Neurony

**Neuron** w tym kontekście to fragment warstwy, który łączy się ze wszystkimi neuronami w poprzedniej warstwie i na ich podstawie produkuje jedno z wyjść warstwy. 

Jeśli nasza warstwa ma postać $g(W^T x + \mathbf{b})$, to $i$-ty neuron implementuje funkcję:
$$ f(x) = g(w_i^T x + b_i), $$
gdzie wektor $w_i$ jest $i$-tą kolumną macierzy $W$ a $b_i$ jest $i$-tym elementem wektora $\mathbf{b}$.

## Zadanie 2 (3 pkt.)
Za pomocą przygotowanego przez TensorFlow [narzędzia do zabawy z sieciami neuronowymi](http://playground.tensorflow.org) proszę przeprowadzić poniższe eksperymenty i opisać rezultaty.

Kilka uwag:
* Każda odpowiedź powinna być zawarta w jednym/dwóch zdaniach. 
* Punktowane będą nie tylko prawidłowe odpowiedzi, ale też sensowne hipotezy/przypuszczenia.
* Jeżeli proszeni są Państwo o podanie architektury sieci, to najlepiej zapisać ją w skrótowe postaci `n_1-n_2-...-n_k`, gdzie `n_i` to liczba neuronów w `i`-tej warstwie. Czyli jeśli sieć ma pięć neuronów w pierwszej warstwie, trzy neurony w drugiej warstwie oraz sześć neuronów w trzeciej warstwie, to można ją opisać jako `5-3-6`.
* Proszę nie zmieniać opcji noise/ratio of training to test data. Proszę nie zmieniać feature'ów wejściowych, o ile nie będzie to wyraźnie podane w zadaniu.
* Można użyć opcji "show test data", żeby sprawdzić, dlaczego koszt na datasecie treningowym i testowym się różni.

1) **Eksperymenty na zbiorze Gaussian**
* Czy ten dataset można rozwiązać metodami płytkimi, których uczyliśmy się na wcześniejszych zajęciach?
* Co sprawia, że ten zbiór danych jest łatwiejszy niż pozostałe?
* Porównaj na tym zbiorze dwa modele: sieć neuronową z kilkoma warstwami i kilkudziesięcioma neuronami oraz sieć z jednym neuronem. Który z tych modeli bardziej nadaje się do zadania?

1. Tak, bo zbiory wyraźnie odseparowane. Dla tego konkretnego przykładu wystarczyłoby użyć k-mean 
albo wskazać funkcję liniową separującą oba zbiory.
2. Jak wspomniane w pierwszym, zbiory danych są wyraźnie odseparowane, a najwięcej punktów jest cantrum zbioru
(im bliżej centrum/centroidu zbioru tym większa pewność co do przynależności).
3. Dla sieci 6-4-6-3-4 rozróżnienie jest w 8 krokach i właście potem się nie zmienia. Dla sieci 7-6-7-3-4 sieć
rozróżnia w 5 krokach. Dla sieci złożonej z jednego neuronu rozróżnienie jest po 6 krokach. Dla zbiory Gaussian lepsza
wydaje się sieć złożona z jedego neurona, wprowadzenie większej ich ilości (i większej ilości warstw) nie zmienia 
istotnie wyniku.

2) **Eksperymenty na zbiorze Circle**
* Załóżmy, że mamy sieć z jednym neuronem. Ile najmniej potrzeba feature'ów wejściowych, żeby model osiągał na datasecie testowym koszt $\leq 0.001$? Jakie to feature'y?
* Załóżmy, że na wejściu mamy tylko niezanurzone feature'y (tzn. $x_1$ oraz $x_2$). Stwórz najmniejszą sieć neuronową (pod względem liczby neuronów), która osiąga na datasecie testowym koszt $\leq 0.001$. Opisz architekturę tej sieci. 
* Spróbuj rozwiązać ten problem za pomocą dowolnie dużej sieci neuronowej **z aktywacjami liniowymi** (nie zmieniając feature'ów wejściowych). Czy udało się osiągnąć wynik $\leq 0.001$? Jeśli tak, podaj architekturę sieci. Jeśli nie, podaj hipotezę, dlaczego się nie udało.

1. Dla sieci z jednym neuronem udało mi się znaleźć rozwiązanie z 4 featurami X2, X12, X22, X1X2. 
Po 150 krokach koszt jest ≤ 0.001
Dla sieci z jednym neuronem udało mi się znaleźć rozwiązanie z 3 featurami X12, X22, X1X2. 
Po 100 krokach koszt jest ≤ 0.001
Dla sieci z jednym neuronem udało mi się znaleźć rozwiązanie z 2 featurami X12, X22. 
Po 110 krokach koszt jest ≤ 0.001

2. Sieć jednowarstwowa z trzema neuronami daje koszt poniżej 0.001 ale po 5,400 kroków.
Z kolei sieć o kształcie 3-2 osiąga koszt ≤ 0.001 po 313 krokach.
Druga warstwa (2 dodatkowe neurony) istotnie przyspieszyła proces klasyfikacji. Wydaje się też, że dla niezanurzonych
featurów nie da się zejść poniżej 3 neuronów, bo uzyskanie "kółka" na środku zbioru staje się niemożliwe.

3. Dla sieci z aktywacjami liniowymi sieć podaje podobne rozwiązanie dla architektury jednowarstwowej z 2 neuronami i dla architektury 8-6-7-5-6. Wydaje się, że jest tak dlatego, że liniowo nie jesteśmy w stanie opisać dystrybucji danych ze zbioru "Circle". Zauważalną cechą sieci dla tego zbioru jest to, że funkcja kosztu dla zbiorów treningowych i testowych rozbiega się po pewnej ilości kroków (training loss maleje, a test loss rośnie, potem się zatrzymują.

3) **Eksperymenty na zbiorze Spiral**
* Osiągnij (stabilny) koszt $\leq 0.1$ na zbiorze testowym, podaj wykorzystaną architekturę, rodzaj aktywacji, regularyzację  oraz learning rate.
* Co odróżnia rozwiązania które dobrze generalizują od rozwiązań, które overfitują pod względem wizualnym? Popatrz na płaszczyznę z danymi po wytrenowaniu modelu.

1. Dla  architektury 5-5-4-3 udało się osiągnąć stabily koszt ≤0.1 (Test loss ~0.042, Training loss 0.016) użyta funkcja
aktywacji to ReLU, regularyzacja L2, Learning rate: 0.003.
Dla tej samej architektury wyższy (0.01) learning rate pomógł wyeliminować overfitting i dał lepszy wynik: 
     Test loss 0.002
     Training loss 0.007

2. Dla zbioru spiral overfitowanie widać szczególnie w przypadkach kiedy generalizacja przebiega przez linię spirali 
(co oznacza, że dla innego zbioru, który akurat w tym punkcie miałby dane model by się mylił.)

## Zadanie 3 (2 pkt.)

Ręcznie zaimplementować prostą sieć z jedną warstwą ukrytą. Sieć:
1. Na wejściu będzie przyjmować dane o wymiarze `input_dim`
2. Pierwsza warstwa ma je przetwarzać na wymiar `hidden_dim`.
3. Druga warstwa ma przetwarzać wyjście pierwszej warstwy na wymiar `output_dim`.

W tym celu trzeba stworzyć odpowiednie tensory reprezentujące wagi i biasy w poszczególnych warstwach
1. Macierze wag należy zainicjalizować za pomocą wartości wylosowanych ze standardowego rozkładu normalnego. 
2. Dla obu warstw należy stworzyć _biasy_ zainicjalizowane na 0.
3. Funkcją aktywacji dla pierwszej warstwy ma być `torch.tanh`. W drugiej warstwie ma być aktywacja liniowa (czyli brak aktywacji).

Następnie należy zaimplementować pętlę uczenia z użyciem PyTorchowej funkcji kosztu `nn.CrossEntropyLoss` i optymalizatora SGD. Jeśli wszystko zostało zaimplementowane poprawne, to sieć powinna zazwyczaj osiagać accuracy większe niż `0.82` na zbiorze testowym (chociaż czasami może nie osiągać tej wartości z powodu pechowej inicjalizacji).

**HINT** Proszę nie zapomnieć o `requires_grad=True` przy definiowaniu parametrów sieci.

In [18]:
from typing import List

class CustomNetwork(object):
    """
    Simple 1-hidden layer linear neural network
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Initialize network's weights 
        """
        
        self.weight_1: torch.Tensor = torch.empty(input_dim).normal_()
        self.weight_1.requires_grad = True
        self.bias_1: torch.Tensor = torch.nn.init.zeros_(torch.empty(input_dim))
        self.bias_1.requires_grad = True
        
        self.weight_2: torch.Tensor = torch.empty(input_dim).normal_()
        self.weight_2.requires_grad = True
        self.bias_2: torch.Tensor = torch.nn.init.zeros_(torch.empty(input_dim))
        self.bias_2.requires_grad = True
        
    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the network
        """

        x = torch.unsqueeze(x, hidden_dim)
        x = torch.tanh(x)
        return torch.unsqueeze(x, output_dim)
        
    def parameters(self) -> List[torch.Tensor]:
        """
        Returns all trainable parameters 
        """
        return [self.weight_1, self.bias_1, self.weight_2, self.bias_2]

In [20]:
from torch import nn
from torch.optim import SGD
from torch.nn.functional import cross_entropy

# some hyperparams
batch_size: int = 64
epoch: int = 3


# prepare data loaders, based on the already loaded datasets
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# initialize the model
model: CustomNetwork = CustomNetwork(5,4,3)

# initialize the optimizer using the hyperparams below
lr: float = 0.01
momentum: float = 0.9
optimizer: torch.optim.Optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

criterion = nn.CrossEntropyLoss()

# training loop
for e in range(epoch):
    for i, (x, y) in enumerate(train_loader):
        # reset the gradients from previouis iteration
        optimizer.zero_grad()
        # pass through the network
        output: torch.Tensor = model()
        # calculate loss
        loss: torch.Tensor = criterion(output)
        # backward pass thorught the network
        loss.backward()
        # apply the gradients
        optimizer.step()
        
        # log the loss value
        if (i + 1) % 100 == 0:
            print(f"Epoch {e} iter {i+1}/{len(train_data) // batch_size} loss: {loss.item()}", end="\r")
            
    # at the end of an epoch run evaluation on the test set
    with torch.no_grad():
        # initialize the number of correct predictions
        correct: int = 0 
        for i, (x, y) in enumerate(test_loader):
            # pass through the network
            output: torch.Tensor = model()
            # update the number of correctly predicted examples
            correct += len(output)

        print(f"\nTest accuracy: {correct / len(test_data)}")

        
# this is your test
assert correct / len(test_data) > 0.82, "Subject to random seed you should be able to get >82% accuracy"

NameError: name 'mean' is not defined